In [1]:
!pip install darts -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build

# Import de bibliotecas

In [2]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 

from datetime import datetime
from itertools import product

from unicodedata import normalize

from darts import TimeSeries, concatenate
from darts.datasets import AustralianTourismDataset
from darts.models import LinearRegressionModel, Theta
from darts.metrics import mae
from darts.dataprocessing.transformers import MinTReconciliator
from darts.dataprocessing import Pipeline
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis
from darts.utils.timeseries_generation import linear_timeseries
from darts.models.forecasting.xgboost import XGBModel
import time
from darts.metrics import rmse
from darts.models.forecasting.random_forest import RandomForest
from darts.models import  AutoARIMA
from darts.metrics import mape

from darts.metrics import coefficient_of_variation as coef_var


# Carregamento de dados: 20/04/2023

In [3]:
def salvar_arquivo(file_to_save,kind:str):
    PATH = f'/content/drive/MyDrive/Dados/standarized/{kind}/' + kind + '_' + datetime.strftime(datetime.today(), '%d-%m-%Y')+'.csv'
    file_to_save.to_csv(PATH, index=False)

In [4]:
df = pd.read_parquet('/content/drive/MyDrive/Dados/refined/table_full.parquet')

# Filtragem de dados:

Produto|Variavel
-------|--------
Abacaxi|Produto das lavouras temporárias
Amendoim|Produto das lavouras temporárias
Batata-Doce|Produto das lavouras temporárias
Feijão|Produto das lavouras temporárias
Melancia|Produto das lavouras temporárias
Milho|Produto das lavouras temporárias
Tomate|Produto das lavouras temporárias
Soja|Produto das lavouras temporárias
Bovino|Pecuária


## Lavouras temporárias

Todos os produtos que serão analisados no artigo estão listados aqui. A suas área de coberturas estão classificadas como "Outros", entretanto, será modificado para "Cobertura".

In [5]:
# Produtos de lavouras temporárias
prod_lavouras_temp = ['Abacaxi*', 'Amendoim (em casca)', 'Batata-doce', 'Cebola'
                    'Feijão (em grão)', 'Melancia', 'Milho (em grão)', 'Tomate']

list_concat_temp = []

for i in prod_lavouras_temp:
    df_temp = df.loc[df['produto'] == i]
    list_concat_temp.append(df_temp)

df_lavouras_temp = pd.concat(list_concat_temp)

del list_concat_temp

df_lavouras_temp

,municipio,tipo_producao,variavel,produto,metrica,ano,valor,cod_mun,uf,reg_inter,reg_imed
185,Abaetetuba,Produto das lavouras permanentes,Quantidade produzida,Abacaxi*,Toneladas,1985,NaN,1500107,PA,Belém,Abaetetuba
186,Abaetetuba,Produto das lavouras permanentes,Área plantada,Abacaxi*,,1985,NaN,1500107,PA,Belém,Abaetetuba
187,Abaetetuba,Produto das lavouras permanentes,Área colhida,Abacaxi*,Hectares,1985,NaN,1500107,PA,Belém,Abaetetuba
188,Abaetetuba,Produto das lavouras permanentes,Quantidade produzida,Abacaxi*,Toneladas,1986,NaN,1500107,PA,Belém,Abaetetuba
189,Abaetetuba,Produto das lavouras permanentes,Área plantada,Abacaxi*,,1986,NaN,1500107,PA,Belém,Abaetetuba
...,...,...,...,...,...,...,...,...,...,...,...
561416,Xinguara,Produto das lavouras permanentes,Área plantada,Tomate,Hectares,2020,NaN,1508407,PA,Redenção,Xinguara
561417,Xinguara,Produto das lavouras permanentes,Área colhida,Tomate,Hectares,2020,NaN,1508407,PA,Redenção,Xinguara
561418,Xinguara,Produto das lavouras permanentes,Quantidade produzida,Tomate,Toneladas,2021,NaN,1508407,PA,Redenção,Xinguara
561419,Xinguara,Produto das lavouras permanentes,Área plantada,Tomate,Hectares,2021,NaN,1508407,PA,Redenção,Xinguara


Modificação de produto para "Cobertura".

In [6]:
df_cobertura_lavoura_temp = df.loc[(df['produto'] == 'Outros') & (df['tipo_producao'] == 'Produto das lavouras temporárias')]
df_cobertura_lavoura_temp['produto'] = 'Cobertura'

<ipython-input-6-99035454a237>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cobertura_lavoura_temp['produto'] = 'Cobertura'


## Soja

In [7]:
df_soja = df.loc[df['produto'] == 'Soja (em grão)']

## Pastagem/Bovino

In [8]:
df_bovino = df.loc[df['produto'] == 'Bovino']
df_pasto = df.loc[df['produto'] == 'Pasto']

# Unificação de dados

Tabelas unificadas:

```
[df_bovino, df_cobertura_lavoura_temp, df_lavouras_temp, df_pasto, df_soja]
```

Produtos exatraídos:

Produto|
-------|
Bovino |
Cobertura |
Abacaxi* |
Amendoim (em casca) |
Batata-doce |
Melancia |
Milho (em grão) |
Tomate |
Pasto |
Soja (em grão) |

In [9]:
df2 = pd.concat([df_bovino, df_pasto])
df2['estado'] = 'Pará'
for i in df2.produto.unique():
    print(i,'|')

df2.head()

Bovino |
Pasto |


,municipio,tipo_producao,variavel,produto,metrica,ano,valor,cod_mun,uf,reg_inter,reg_imed,estado
0,Abaetetuba,Pecuária,Efetivo dos rebanhos,Bovino,Cabeças,1985,1428.0,1500107,PA,Belém,Abaetetuba,Pará
1,Abaetetuba,Pecuária,Efetivo dos rebanhos,Bovino,Cabeças,1986,3420.0,1500107,PA,Belém,Abaetetuba,Pará
2,Abaetetuba,Pecuária,Efetivo dos rebanhos,Bovino,Cabeças,1987,3633.0,1500107,PA,Belém,Abaetetuba,Pará
3,Abaetetuba,Pecuária,Efetivo dos rebanhos,Bovino,Cabeças,1988,4864.0,1500107,PA,Belém,Abaetetuba,Pará
4,Abaetetuba,Pecuária,Efetivo dos rebanhos,Bovino,Cabeças,1989,5107.0,1500107,PA,Belém,Abaetetuba,Pará


# Limpeza/Modelagem

In [19]:
df_merge = df2[['municipio', 'reg_imed', 'reg_inter',
                'tipo_producao', 'variavel', 'produto', 'metrica', 'ano', 'valor']]
df_ano = df_merge.groupby('ano', as_index=False).sum()[['ano']].astype('int')

<ipython-input-19-3f81f831cbe9>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_ano = df_merge.groupby('ano', as_index=False).sum()[['ano']].astype('int')


Agrupamento por ano

## Modelagem

In [20]:
df_final = df_ano[['ano']]

drop_columns = list()

dict_plot = dict()
list_plot = list()

list_var = list()



for variav in df_merge.variavel.unique():
    list_prod = list()
    df_temp1 = df_merge.loc[df_merge['variavel'] == variav]
    v_1 = df_temp1.groupby('ano', as_index=False).sum()['valor'].sum()

    
    if v_1 > 1000: 
        # 2.1.1 Normaliza item e armazena em x2
        x_0 = normalize('NFKD', variav).encode('ASCII','ignore').decode('ASCII').upper().replace(' ', '_')
        x_0 = 'VAR_' + x_0
        df_final[x_0] = df_temp1.groupby('ano', as_index=False).sum()['valor']

        # Anexar as variáveis 
        
        list_var.append(x_0)
        

        for prod in df_temp1.produto.unique():
            
            print(prod)
            df_temp2 = df_temp1.loc[df_temp1['produto'] == prod]
            v_2 = df_temp2.groupby('ano', as_index=False).sum()['valor'].sum()
            

            if v_2 > 1000: 
                
                # 2.1.1 Normaliza item e armazena em x2
                x_01 = normalize('NFKD', prod).encode('ASCII','ignore').decode('ASCII').upper().replace(' ','_').replace('(', '').replace(')','')
                x_01 = x_0 + ' - PRODUTO_' + x_01
                list_prod.append(x_01)
                
                # 2.1.4 Agrupa, soma e salva o somatório de regi~pes imediatas em df_final
                df_final[x_01] = df_temp2.groupby('ano', as_index=False).sum()['valor']
                
                
                list_inter = list()              
                for i_inter in df_temp2['reg_inter'].unique():
                    df_temp3 = df_temp2.loc[df_temp2['reg_inter'] == i_inter]
                    v_inter = df_temp3.loc[df_temp3['reg_inter'] == i_inter].groupby('ano', as_index=False).sum()['valor'].sum() 
                    print(v_inter)
                    if v_inter > 1000: 
                        # 1.2.1 Normaliza e colocar em capital, amazena em {x}
                        x = normalize('NFKD', i_inter).encode('ASCII','ignore').decode('ASCII').upper()
                        
                        x = x_01 + ' - REGINTER_' + x
                        list_inter.append(x)
                        
                        
                                # 1.2.2 Sobreescreve x de 1.2.1 com o préfixo de {INTER_} + {x}
                            # 1.2.3 Retorna para df_final salvando na coluna o somatório de soja das regiões intermediárias
                            #       onde i_inter = Regiões intemediárias & x = nome das colunas no df_final{Início}
                            
                        df_final[x] = df_temp3.loc[df_temp3['reg_inter'] == i_inter].groupby('ano', as_index=False).sum()['valor']
                            # 1.2.4 Registra temporariamente em df_temp os registros com base na região
                            #       intermediária descrita em 1.2
                        
                        

                        # ----------------------------- Região Imediata --------------------------------
                        list_imed = list()
                        # 2.1 Para cada {i_imed} em {df_temp}{1.4.2}
                        for i_imed in df_temp3['reg_imed'].unique():
                            df_temp4 = df_temp3.loc[df_temp3['reg_imed'] == i_imed]
                            v_imed = df_temp4.groupby('ano', as_index=False).sum()['valor'].sum()
                            print(v_imed)
                            if v_imed > 1000: 
                                # 2.1.1 Normaliza item e armazena em x2
                                x2 = normalize('NFKD', i_imed).encode('ASCII','ignore').decode('ASCII').upper().replace(' ', '').replace('-','')
                                x2 = x + ' - REGIMED_' + x2
                                list_imed.append(x2)
                                # 2.1.4 Agrupa, soma e salva o somatório de regi~pes imediatas em df_final
                                df_final[x2] = df_temp4.groupby('ano', as_index=False).sum()['valor']
                            else:
                                drop_columns.append(x2)

                        dict_plot[x] = list_imed
                dict_plot[x_01] = list_inter                    
        dict_plot[x_0] = list_prod                         
dict_plot['TOTAL'] = list_var


for i in df_final.columns:
    if df_final[i].sum() == 0:
        df_final = df_final.drop(columns = [i])

df_final['TOTAL'] = df_final[list_var].sum(axis=1)

<ipython-input-20-8a431d88185e>:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  v_1 = df_temp1.groupby('ano', as_index=False).sum()['valor'].sum()
<ipython-input-20-8a431d88185e>:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_final[x_0] = df_temp1.groupby('ano', as_index=False).sum()['valor']
<ipython-input-20-8a431d88185e>:33: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  v_2 = df_temp2.groupby('ano', as_index=F

Bovino
11812227.0
4650512.0
6957266.0
204449.0
137097688.0
78491494.0
37249252.0
21356942.0
16587122.0
5835643.0
10751479.0
178689941.0
47216034.0
75659400.0
55814507.0
58275440.0
19027314.0
28063052.0
11185074.0
41113755.0
1803055.0
39310700.0
47276986.0
6108924.0
26970004.0
2159715.0


<ipython-input-20-8a431d88185e>:75: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  v_imed = df_temp4.groupby('ano', as_index=False).sum()['valor'].sum()
<ipython-input-20-8a431d88185e>:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_final[x2] = df_temp4.groupby('ano', as_index=False).sum()['valor']
<ipython-input-20-8a431d88185e>:75: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  v_imed = df_temp4.groupby('ano', as_in

4450517.0
7587826.0
Pasto
28822456.34729623
10977779.708820365
15007032.67633136
2837643.9621444996
141867312.67849153
77878781.10219021
42746116.77890296
21242414.79739835
4005673.1685987096
3746519.0324938092
259154.1361049017
161759598.8713811
37042718.92277544
81985630.01329954
42731249.93530615
69103034.0679963
26380493.247118838
32553049.24138064
10169491.579496827
46879679.74215111


<ipython-input-20-8a431d88185e>:64: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_final[x] = df_temp3.loc[df_temp3['reg_inter'] == i_inter].groupby('ano', as_index=False).sum()['valor']
<ipython-input-20-8a431d88185e>:75: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  v_imed = df_temp4.groupby('ano', as_index=False).sum()['valor'].sum()
<ipython-input-20-8a431d88185e>:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  d

5918157.871467346
40961521.87068376
86694907.95045719
13576462.766291538
40324925.55093424
4890016.023611641
12735175.911715515
15168327.697904233


<ipython-input-20-8a431d88185e>:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_final[x2] = df_temp4.groupby('ano', as_index=False).sum()['valor']
<ipython-input-20-8a431d88185e>:75: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  v_imed = df_temp4.groupby('ano', as_index=False).sum()['valor'].sum()
<ipython-input-20-8a431d88185e>:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_final[x2] = df_temp4.groupby('ano',

In [21]:
df_final = df_final + (0.00001* np.random.rand(df_final.shape[0], df_final.shape[1]))
df_final['ano'] = df_ano['ano']

In [22]:
df_final['TOTAL'] = df_final[list_var].sum(axis=1)

In [23]:
df_final

,ano,VAR_EFETIVO_DOS_REBANHOS,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_ABAETETUBA,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_BELEM,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_CAMETA,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_MARABA,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_TUCURUI,...,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_ALTAMIRA,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_ALTAMIRA - REGIMED_ALMEIRIMPORTODEMOZ,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_ALTAMIRA - REGIMED_ALTAMIRA,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL - REGIMED_BRAGANCA,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL - REGIMED_PARAGOMINAS,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL - REGIMED_CAPANEMA,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL - REGIMED_CAPITAOPOCO,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL - REGIMED_CASTANHAL,TOTAL
0,1985,3.446825e+06,3.446825e+06,142483.000006,37341.000004,102579.000006,2563.000001,6.499120e+05,5.968720e+05,5.304000e+04,...,4.425032e+05,159379.353143,2.831238e+05,1.424296e+06,178439.113035,7.490575e+05,69335.681485,199569.909047,227894.276252,9.684344e+06
1,1986,4.102298e+06,4.102298e+06,192805.000009,73342.000009,116217.000003,3246.000002,6.517330e+05,5.785600e+05,7.317300e+04,...,4.129635e+05,142983.100455,2.699804e+05,1.407399e+06,178332.039269,7.472007e+05,61748.917870,199242.179687,220874.713699,1.015863e+07
2,1987,4.641462e+06,4.641462e+06,211405.000008,76995.000002,129179.000008,5231.000007,7.926560e+05,7.147550e+05,7.790100e+04,...,4.248910e+05,116114.893577,3.087761e+05,1.524108e+06,182486.759709,7.893329e+05,69334.774575,214312.782889,268640.864448,1.121351e+07
3,1988,5.431061e+06,5.431061e+06,232784.000008,96694.000001,130997.000005,5093.000002,7.806910e+05,6.963000e+05,8.439100e+04,...,4.091791e+05,89350.653630,3.198285e+05,1.648043e+06,203419.592146,8.237939e+05,83779.824748,235400.105074,301649.152425,1.248444e+07
4,1989,5.955474e+06,5.955474e+06,278257.000005,143607.000007,129457.000003,5193.000006,1.291445e+06,8.563690e+05,2.889760e+05,...,4.087488e+05,70753.193144,3.379956e+05,1.589743e+06,185340.019877,8.049924e+05,85797.164490,222275.173538,291338.314746,1.315114e+07
5,1990,6.292421e+06,6.292421e+06,282747.000003,148540.000009,129242.000004,4965.000002,1.363288e+06,9.089880e+05,2.983000e+05,...,4.000584e+05,50572.330111,3.494861e+05,1.617110e+06,185094.461495,8.313490e+05,78769.373978,242129.942075,279767.198851,1.378513e+07
6,1991,6.714444e+06,6.714444e+06,285850.000002,151700.000002,129589.000009,4561.000004,1.372751e+06,9.172610e+05,3.016900e+05,...,3.919395e+05,48913.072443,3.430264e+05,1.757746e+06,279786.011889,8.369256e+05,83939.471256,261676.954120,295418.334106,1.456283e+07
7,1992,7.066588e+06,7.066588e+06,316478.000003,151450.000002,161022.000001,4006.000006,1.489186e+06,1.001462e+06,3.265240e+05,...,4.505395e+05,68006.704108,3.825328e+05,1.835856e+06,274843.050910,8.739700e+05,100457.447473,259280.383150,327304.970724,1.558026e+07
8,1993,7.515580e+06,7.515580e+06,371707.000005,153500.000009,214028.000005,4179.000003,1.403395e+06,8.837900e+05,3.896050e+05,...,5.221311e+05,99036.892308,4.230942e+05,1.868231e+06,273750.739203,8.922308e+05,106195.486660,255781.599571,340272.052526,1.664394e+07
9,1994,7.624234e+06,7.624234e+06,392198.000008,158530.000009,229320.000005,4348.000001,1.393568e+06,8.832800e+05,3.721880e+05,...,6.026589e+05,158657.703272,4.440012e+05,1.832936e+06,273635.769405,8.999090e+05,99042.276785,251322.873174,309026.389914,1.722632e+07


In [25]:
dict_plot

{'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM': ['VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_ABAETETUBA',
  'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_BELEM',
  'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_CAMETA'],
 'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA': ['VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_MARABA',
  'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_TUCURUI',
  'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_PARAUAPEBAS'],
 'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BREVES': ['VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BREVES - REGIMED_BREVES',
  'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BREVES - REGIMED_SOURESALVATERRA'],
 'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_REDENCAO': ['VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_REDENCAO - REGIME

In [26]:
print(f'{"="*50}\nNOME DE COLUNAS\n{"="*50}\n\n')
for i in df_final.columns:
    print(i,'\n-----------------------')
    print(df_final[i].sum(),'\n')

NOME DE COLUNAS


ano 
-----------------------
74111 

VAR_EFETIVO_DOS_REBANHOS 
-----------------------
490853159.00018615 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO 
-----------------------
490853159.0001786 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM 
-----------------------
11812227.000186188 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_ABAETETUBA 
-----------------------
4650512.0002179565 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_BELEM 
-----------------------
6957266.000180322 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_CAMETA 
-----------------------
204449.0001893706 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA 
-----------------------
137097688.0002158 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_MARABA 
-----------------------
78491494.00019176 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_TUCURUI 
---------

In [27]:
print(f'{"="*50}\nNOME DE COLUNAS = 0 \n{"="*50}\n\n')
for i in df_final.columns:
    if df_final[i].sum() == 0:
        print(i,'\n-----------------------')
        print(df_final[i].sum(),'\n')


NOME DE COLUNAS = 0 




In [28]:
salvar_arquivo(df_final, 'time_series')

# Criação de hierarquias

In [29]:
hierarchy = dict()
# Itens que não devem ser iterados (totais)

# Para cada Item nas colunas:
for i in df_final.columns:
    # Condicional
    if i != 'ano' and i != 'TOTAL':

        # Text_split = i (nome da coluna) separado por " - "
        text_split = i.split(' - ')
        # Lenf é igual ao tamanho do texto separado (nível hierarquico)
        lenf = len(text_split)
        # Nova lista que irá conter as suas hierarquias superiores
        new_list = [] 

        # Enquanto o contador (tamanho do texto separado) for maior que 1:
        while lenf > 1:
            # Debugger
            new_list.append(' - '.join(text_split[0:lenf-1]))
            lenf-= 1
        hierarchy[i] = new_list
        
keys = []
for k, v in hierarchy.items():
    if len(v) == 0:
        hierarchy[k] = ['TOTAL']

for i in keys:
    if i not in df_final.columns:
        print(i)

In [30]:
with open(f'/content/drive/MyDrive/Dados/standarized/dict_plot/dict_plot_{datetime.strftime(datetime.today(), "%d-%m-%Y")}.pkl', 'wb') as fp:
    pickle.dump(dict_plot, fp)


In [31]:
with open(f'/content/drive/MyDrive/Dados/standarized/hierarchy/hierarchy_{datetime.strftime(datetime.today(), "%d-%m-%Y")}.pkl', 'wb') as fp:
    pickle.dump(hierarchy, fp)


In [32]:
time_series = TimeSeries.from_dataframe(df =df_final, time_col= 'ano', value_cols=df_final.columns.drop('ano'), hierarchy=hierarchy)

In [33]:
time_series

<TimeSeries (DataArray) (ano: 37, component: 61, sample: 1)>
array([[[ 3446825.00000741],
        [ 3446825.00000808],
        [  142483.00000612],
        ...,
        [  199569.90904674],
        [  227894.27625241],
        [ 9684343.96101824]],

       [[ 4102298.00000453],
        [ 4102298.00000127],
        [  192805.00000935],
        ...,
        [  199242.17968666],
        [  220874.7136989 ],
        [10158628.05381447]],

       [[ 4641462.00000739],
        [ 4641462.00000074],
        [  211405.00000818],
        ...,
...
        ...,
        [  400289.75026844],
        [  522039.41882887],
        [41146735.13031432]],

       [[22473948.00000241],
        [22473948.00000479],
        [  364142.00000202],
        ...,
        [  393523.74112712],
        [  520379.14672812],
        [42798156.6676991 ]],

       [[23964405.0000097 ],
        [23964405.00000659],
        [  380961.00000581],
        ...,
        [  386913.0313743 ],
        [  512944.50207332],
        [45087010.39906415]]])
Coordinates:
  * ano        (ano) int64 1985 1986 1987 1988 1989 ... 2017 2018 2019 2020 2021
  * component  (component) object 'VAR_EFETIVO_DOS_REBANHOS' ... 'TOTAL'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          {'VAR_EFETIVO_DOS_REBANHOS': ['TOTAL'], 'VAR_EFETIVO_...